In [0]:
"""
A simple notebook showing data analysis and data manipulation using pyspark.
Notebook exported from databricks cloud.
If needed, change file location to local directory under 'files' folder.
"""

In [0]:
# RDD SECTION

In [0]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("students")
sc = SparkContext.getOrCreate(conf=conf)

In [0]:
file1 = "/FileStore/tables/StudentData.csv"
rdd = sc.textFile(file1)
headers = rdd.first()
rdd = rdd.filter(lambda x: x != headers)
rdd = rdd.map(lambda x: x.split(","))

In [0]:
# total number of students

rdd.count()

Out[21]: 1000

In [0]:
# total marks by students by gender

rdd2 = rdd
rdd2.map(lambda x: (x[1], int(x[5])) ).reduceByKey(lambda x,y: (x+y) ).collect()

Out[28]: [('Female', 29636), ('Male', 30461)]

In [0]:
# average mark by students by gender

rdd3 = rdd
rdd3.map(lambda x: (x[1], (int(x[5]), 1) )).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]) ).map(lambda x: (x[0], round(x[1][0]/x[1][1],0)) ).collect()

Out[40]: [('Female', 59.0), ('Male', 61.0)]

In [0]:
# total mumber of students who passed and failed (50+ points to pass)

rdd4 = rdd
passed = rdd4.map(lambda x: int(x[5]) ).filter(lambda x: (x>50) ).count()
failed = rdd4.map(lambda x: int(x[5]) ).filter(lambda x: (x<=50) ).count()

print(f"{passed} students have passed")
print(f"{failed} students have passed")

630 students have passed
370 students have passed


In [0]:
# total enrollment per course

rdd5 = rdd
rdd5.map(lambda x: (x[3],1) ).reduceByKey(lambda x,y: (x+y)).collect()

Out[56]: [('DB', 157),
 ('Cloud', 192),
 ('PF', 166),
 ('MVC', 157),
 ('OOP', 152),
 ('DSA', 176)]

In [0]:
# total marks per course

rdd6 = rdd
rdd6.map(lambda x: (x[3],int(x[5])) ).reduceByKey(lambda x,y: (x+y)).collect()

Out[59]: [('DB', 9270),
 ('Cloud', 11443),
 ('PF', 9933),
 ('MVC', 9585),
 ('OOP', 8916),
 ('DSA', 10950)]

In [0]:
# average marks per course

rdd7 = rdd
rdd7.map(lambda x: (x[3],(int(x[5]),1)) ).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0], round(x[1][0]/x[1][1],0)) ).collect()
# mapValues
# rdd7.map(lambda x: (x[3],(int(x[5]),1)) ).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).mapValues(lambda x: round(x[0]/x[1],0) ).collect()

Out[75]: [('DB', 59.0),
 ('Cloud', 60.0),
 ('PF', 60.0),
 ('MVC', 61.0),
 ('OOP', 59.0),
 ('DSA', 62.0)]

In [0]:
# finding minimum and maximum marks per course

rdd8 = rdd
# minimum mark per course
print("minimum mark per course are: ", rdd8.map(lambda x: (x[3],int(x[5])) ).reduceByKey(lambda x,y: (x if x < y else y) ).collect())
print("maximum mark per course are: ", rdd8.map(lambda x: (x[3],int(x[5])) ).reduceByKey(lambda x,y: (x if x > y else y) ).collect())

minimum mark per course are:  [('DB', 20), ('Cloud', 20), ('PF', 20), ('MVC', 22), ('OOP', 20), ('DSA', 20)]
maximum mark per course are:  [('DB', 98), ('Cloud', 99), ('PF', 99), ('MVC', 99), ('OOP', 99), ('DSA', 99)]


In [0]:
# average age of male and female students

rdd9 = rdd
rdd9.map(lambda x: (x[1], (int(x[0]),1)) ).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).mapValues(lambda x: round(x[0]/x[1],0)).collect()

Out[85]: [('Female', 28.0), ('Male', 29.0)]

In [0]:
# DATAFRAME SECTION

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, round, sum, avg, max, min, mean, count
spark = SparkSession.builder.appName("quiz").getOrCreate()

In [0]:
file2 = "/FileStore/tables/StudentData.csv" # change to local directory if needed
df = spark.read.options(inferSchema=True, header=True).csv(file2)
df.show()
df.printSchema()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 

In [0]:
# creating new column 'total marks'

df = df.withColumn("total marks", lit(120))
df.show()

+---+------+----------------+------+------+-----+--------------------+-----------+
|age|gender|            name|course|  roll|marks|               email|total marks|
+---+------+----------------+------+------+-----+--------------------+-----------+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...|        120|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|        120|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|        120|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|        120|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|        120|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|        120|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|        120|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|        120|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|        120|
| 29

In [0]:
# creating new column 'average marks' where avg = df.marks/df.total marks *100

df = df.withColumn("average", round((col("marks")/col("total marks"))*100,2))
df.show()

+---+------+----------------+------+------+-----+--------------------+-----------+-------+
|age|gender|            name|course|  roll|marks|               email|total marks|average|
+---+------+----------------+------+------+-----+--------------------+-----------+-------+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...|        120|  49.17|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|        120|  51.67|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|        120|   37.5|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|        120|  24.17|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|        120|  34.17|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|        120|  26.67|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|        120|   57.5|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|        120|  70.83|

In [0]:
# students who passed OOP course with a min 80% score

df_OOP = df.filter((df.course == "OOP") & (df.average > 80))
df_OOP.show()

+---+------+------------------+------+-------+-----+--------------------+-----------+-------+
|age|gender|              name|course|   roll|marks|               email|total marks|average|
+---+------+------------------+------+-------+-----+--------------------+-----------+-------+
| 28|  Male|    Jenna Montague|   OOP|3331161|   98|Leontine Phillips...|        120|  81.67|
| 29|Female|Priscila Tavernier|   OOP|3902993|   99|Celeste Lollis_Bi...|        120|   82.5|
| 28|Female|      Judie Chipps|   OOP|5451977|   99|Tamera Blakley_Mi...|        120|   82.5|
| 29|  Male|    Margene Moores|   OOP|5621072|   97|Sheryll Towler_Ma...|        120|  80.83|
| 29|  Male|      Jc Andrepont|   OOP|8022618|   97|Cordie Harnois_Ja...|        120|  80.83|
| 28|  Male|    Loris Crossett|   OOP|8172914|   98|Paris Hutton_Pari...|        120|  81.67|
| 28|  Male|    Loris Crossett|   OOP|9692316|   99|Judie Chipps_Mich...|        120|   82.5|
+---+------+------------------+------+-------+-----+--------

In [0]:
# students who passed Cloud course with a min 60% score

df_Cloud = df.filter((df.course == "Cloud") & (df.average > 60))
df_Cloud.show()

+---+------+-----------------+------+-------+-----+--------------------+-----------+-------+
|age|gender|             name|course|   roll|marks|               email|total marks|average|
+---+------+-----------------+------+-------+-----+--------------------+-----------+-------+
| 28|Female|     Claude Panos| Cloud|  72409|   85|Sheryll Towler_Al...|        120|  70.83|
| 29|  Male|      Billi Clore| Cloud| 512047|   76|Taryn Brownlee_Ju...|        120|  63.33|
| 28|Female|   Somer Stoecker| Cloud| 612490|   82|Sebrina Maresca_G...|        120|  68.33|
| 29|Female|     Judie Chipps| Cloud| 632793|   75|Tijuana Kropf_Ele...|        120|   62.5|
| 29|Female|     Eda Neathery| Cloud|1011971|   91|Margene Moores_El...|        120|  75.83|
| 28|  Male|   Bonita Higuera| Cloud|1312294|   94|Eda Neathery_Pris...|        120|  78.33|
| 29|Female|  Hubert Oliveras| Cloud|1392791|   94|Anna Santos_Alber...|        120|  78.33|
| 28|Female|      Neda Briski| Cloud|1651303|   74|Ernest Rossbach_D..

In [0]:
# grouping by gender and course and getting count, min, max, avg, sum and totals of marks

df.groupBy("gender", "course").agg(count("*").alias("total_enrollment"), sum("marks").alias("total_marks"), max("marks").alias("max_marks"),  min("marks").alias("min_marks"), round(avg("marks"),0).alias("avg_marks")).show()

+------+------+----------------+-----------+---------+---------+---------+
|gender|course|total_enrollment|total_marks|max_marks|min_marks|avg_marks|
+------+------+----------------+-----------+---------+---------+---------+
|Female| Cloud|             106|       6316|       99|       20|     60.0|
|Female|   OOP|              82|       4682|       99|       21|     57.0|
|  Male|    PF|              97|       5960|       99|       20|     61.0|
|  Male|    DB|              82|       5073|       98|       20|     62.0|
|  Male| Cloud|              86|       5127|       97|       21|     60.0|
|  Male|   OOP|              70|       4234|       99|       20|     60.0|
|Female|   DSA|              98|       6124|       99|       20|     62.0|
|Female|    DB|              75|       4197|       96|       20|     56.0|
|  Male|   DSA|              78|       4826|       99|       20|     62.0|
|  Male|   MVC|              86|       5241|       99|       22|     61.0|
|Female|   MVC|          

In [0]:
# total number of students in each course

df.groupBy("course").count().show()

+------+-----+
|course|count|
+------+-----+
|    PF|  166|
|    DB|  157|
|   MVC|  157|
|   DSA|  176|
| Cloud|  192|
|   OOP|  152|
+------+-----+



In [0]:
# total number of students in each course (male / female)

df.groupBy("gender", "course").count().show()

+------+------+-----+
|gender|course|count|
+------+------+-----+
|Female| Cloud|  106|
|Female|   OOP|   82|
|  Male|    PF|   97|
|  Male|    DB|   82|
|  Male| Cloud|   86|
|  Male|   OOP|   70|
|Female|   DSA|   98|
|Female|    DB|   75|
|  Male|   DSA|   78|
|  Male|   MVC|   86|
|Female|   MVC|   71|
|Female|    PF|   69|
+------+------+-----+



In [0]:
# total marks for each gender in each course

df.groupBy("gender","course").agg(sum("marks")).show()

+------+------+----------+
|gender|course|sum(marks)|
+------+------+----------+
|Female| Cloud|      6316|
|Female|   OOP|      4682|
|  Male|    PF|      5960|
|  Male|    DB|      5073|
|  Male| Cloud|      5127|
|  Male|   OOP|      4234|
|Female|   DSA|      6124|
|Female|    DB|      4197|
|  Male|   DSA|      4826|
|  Male|   MVC|      5241|
|Female|   MVC|      4344|
|Female|    PF|      3973|
+------+------+----------+



In [0]:
# min, max, avg marks for each age by course

df.groupBy("age", "course").agg(min("marks").alias("min_marks"), max("marks").alias("max_marks"), round(avg("marks"),1).alias("avg_marks")).show()

+---+------+---------+---------+---------+
|age|course|min_marks|max_marks|avg_marks|
+---+------+---------+---------+---------+
| 29|   OOP|       20|       99|     59.7|
| 28|    PF|       20|       98|     63.8|
| 28|    DB|       21|       98|     58.8|
| 28|   DSA|       20|       99|     64.7|
| 29|    DB|       20|       98|     59.3|
| 28| Cloud|       20|       99|     58.1|
| 29|   DSA|       20|       99|     60.0|
| 28|   MVC|       23|       99|     60.4|
| 28|   OOP|       23|       99|     57.6|
| 29|    PF|       20|       99|     56.3|
| 29|   MVC|       22|       99|     61.6|
| 29| Cloud|       21|       98|     61.3|
+---+------+---------+---------+---------+

